# Parametric building frame in reinforced concrete

In this workshop the goal is to write a function that takes in input a string that is the name of a file that contains, on the odd lines, 3D vector positioning the local origin of the next frame with respect to the local origin of the previous one and, on the even lines, the actual parameters of a planar concrete frame.

The goal frame is shown in the figure below (we are not considering foundations):

![Image of the structure](struct.jpg)

using this as reference system: ![Image of the rs](rgb.png)

The program builds continuous pillars successively intersected by beams, so it requires 3D coordinates to start assembling the frames. The following coordinates are given by considering the previous coordinates as the center. The program also requires actual parameters to build the frame, that are beams dimensions, pillar dimensions and distances and interstory heights. Those values are given as an input file that is shown below, having coordinates on odd lines and other parameters on the even rows (with the above enlisted order):

```0;0;0
(.4,.4);(.8,.8);[-4.,-4.,-4.,-4.,-4.,-4.];[-3,-4,-4,-4,-4,-4]
5;0;0
(.4,.4);(.8,.8);[-4.,-4.,-4.,-4.,-4.,-4.];[-3,-4,-4,-4,-4,-4]
5;0;0
(.4,.4);(.8,.8);[-4.,-4.,-4.,-4.,-4.,-4.];[-3,-4,-4,-4,-4,-4]```

There is a main function ***ggpl_bone_structure*** that takes in input the path of this file as a string, builds each frame reading 2 lines of the file and then the connecting beams:


In [11]:
def ggpl_bone_structure(file_name):
	"""This function takes in input a file_name string that is the path of a file that contains 3D coordinates and parameters for a planar frame
	and returns the VIEW of the STRUCT required."""
	with open(file_name, 'rb') as file:
		reader = csv.reader(file, delimiter = ';')
		finalModel = []
		xCoord = 0
		yCoord = 0 
		zCoord = 0
		fileValues = []
		rowAcc = 0
		for row in reader:
			rowAcc += 1
			fileValues.append(row)
			if(rowAcc == 2):
				beamlengthY = []
				xCoord += float(fileValues[0][0])
				yCoord += float(fileValues[0][1])
				zCoord += float(fileValues[0][2])
				frameModel = buildFrame(make_tuple(fileValues[1][0]), make_tuple(fileValues[1][1]), make_tuple(fileValues[1][2]), make_tuple(fileValues[1][3]))
				frameElement = STRUCT([T(1)(xCoord), T(2)(yCoord), T(3)(zCoord), frameModel])
				finalModel.append(STRUCT([frameElement]))
				rowAcc = 0
				fileValues = []
		finalModel.append(buildBeams(file_name))
		VIEW(STRUCT(finalModel))

and it finally returns the ***VIEW*** of the ***STRUCT*** builded.

This function has some support functions, such as an ***intersperse*** function useful to define value vectors for pillars and beams:

In [3]:
def intersperse(seq, value):
    """This function intersperse an input list with the input value. If the resultant list has odd length, another given value is appended
    intersperse is a function that, given a list and a value, intersperse the list with the value."""
    res = [value] * (2 * len(seq) - 1)
    res[::2] = seq
    if (len(res)%2 != 0):
        res.append(value)
    return res

### buildFrame function

This function creates each frame as said above by getting its parameters in input:

In [7]:
def buildFrame(beamDimensions, pillarDimensions, pillarDistances, interstoryHeights):
    """This function returns an HPC model of a concrete space frame with given beam's and pillar's dimensions, 
    distances between the pillars, and interstories."""
 	
    pillarDistances = [0] + pillarDistances
    linearPillars = intersperse(pillarDistances, pillarDimensions[1])
    pillars3D = INSR(PROD)([QUOTE([pillarDimensions[0], -3]),QUOTE(linearPillars), QUOTE(intersperse([-interstory for interstory in interstoryHeights], -beamDimensions[1]))])
    horizontalBeamXYAxis = [pillarDimensions[0],-3]
    horizontalBeamYYAxis = intersperse([-beam for beam in pillarDistances], pillarDimensions[1])
    horizontalBeamYYAxis[0] = -horizontalBeamYYAxis[0]
    beamsY3D = INSR(PROD)([QUOTE(horizontalBeamXYAxis), QUOTE(horizontalBeamYYAxis), QUOTE(intersperse(interstoryHeights,beamDimensions[1]))])
    frameModel = STRUCT([pillars3D, beamsY3D])
    return frameModel

and returns a single frame that is then translated by the 3D coordinates read and **appended** to the *finalModel*:

![Image of a frame](singleFrame.png)

## buildBeams function

The last helper function called is *buildBeams(file_name)* that finally connects the frames in the *finalModel* by reading twice the input file:

In [10]:
def buildBeams(file_name):
	"""This function returns the beams that connect the frames by reading the input file"""
	with open(file_name, 'rb') as file:
		reader = csv.reader(file, delimiter=';')
		beamlengthX = []
		fileValues = []
		rowAcc = 0
		for row in reader:
			rowAcc = rowAcc + 1
			fileValues.append(row)
			if(rowAcc == 2):
				if(float(fileValues[0][0]) == 0):
					beamlengthX.append(-(make_tuple(fileValues[1][1])[0]))
				else:
					beamlengthX.append(float(fileValues[0][0])-make_tuple(fileValues[1][1])[0])
					beamlengthX.append(-(make_tuple(fileValues[1][1])[0]))
				rowAcc = 0
				fileValues = []
	with open(file_name, 'rb') as file:
		reader = csv.reader(file, delimiter=';')
		beamlengthY = []
		beamlengthZ = []
		fileValues = []
		rowAcc = 0
		for row in reader:
			beamlengthY = []
			rowAcc += 1
			fileValues.append(row)
			if(rowAcc == 2):
				beamlengthZ = intersperse(make_tuple(row[3]), make_tuple(row[0])[1])
				beamlengthY.append(make_tuple(fileValues[1][1])[0])
				for element in make_tuple(fileValues[1][2]):
					beamlengthY.append(element)
					beamlengthY.append(make_tuple(fileValues[1][1])[0])
				rowAcc = 0
				fileValues = []
	return INSR(PROD)([QUOTE(beamlengthX), QUOTE(beamlengthY), QUOTE(beamlengthZ)])

The *return* of this function is then appended to the *finalModel* and then the last line of the ***ggpl_bone_structure***:

    VIEW(STRUCT(finalModel))

that gives this result: ![Image of the result](result.png)